In [1]:
import sys
from sklearn import svm
import adversaries as ad
from data_reader import input, output, operations
from classifier_wrapper import Classifier
from evasion_metrics import EvasionMetrics
from copy import deepcopy

#-----------------------------Library Demo---------------------------------
# Demonstrate framework testing by simulating an SVM classifier and its  
# resulting classification accuracy after an attack from the simple_optimize.py adversary.

In [ ]:
# Initial learner training:

# choose a learning model 
learning_model = svm.SVC(probability=True, kernel='linear')

# load the instances to feed to the learner and adversary 
instances = input.load_instances('./data_reader/data/test/100_instance_debug')

# create a naive classifier using classifier_wrapper and specify the training data 
clf = Classifier(learning_model, instances)

#train the classifier on the data
clf.train()


In [ ]:
#Launch attack on the classifier using an AdversarialStrategy:

# choose an adversary attack strategy, here we choose simple_optimize
adv = ad.SimpleOptimize(lambda_val=-100, max_change = 65, learner = clf)

# perform the attack/strategy on the data 
instances_post_attack = adv.attack(instances)

In [ ]:
# obtain metrics of classification performance both pre- and post- attack
metrics = EvasionMetrics(clf, instances, instances_post_attack)
preAccuracy = metrics.getAccuracy(True)
postAccuracy = metrics.getAccuracy(False)
print ("Pre-attack Accuracy (naive classifier): " + str(preAccuracy))
print ("Post-attack Accuracy: (naive classifier): " + str(postAccuracy))

In [ ]:
# plot ROC curve of naive classifier performance after attack
# currently the AUC is 1 because false positive rate is always in the demo case
metrics.plotROC(False)

In [ ]:
# Learner responds by exercising a robust strategy when training:

# create a robust classifier(classifier with an internal mechanism to protect against attack
instances2 = input.load_instances('./data_reader/data/test/100_instance_debug')
learning_model2 = svm.SVC(probability=True, kernel='linear')
clf2 = Classifier(learning_model2, instances2, ["retraining", adv.clone()])
#clf2.set_simulated_adversary_params({'lambda_val': -100, 'max_change': 65})
clf2.train()

In [ ]:
# Now we launch the attack on the robust classifier 

adv = ad.SimpleOptimize(lambda_val=-100, max_change = 65, learner = clf2)



In [ ]:
instances_post_attack2 = adv.attack(instances2)

In [ ]:
# obtain metrics of classification performance both pre- and post- attack
metrics2 = EvasionMetrics(clf2, instances, instances_post_attack2)
preAccuracy2 = metrics2.getAccuracy(True)
postAccuracy2 = metrics2.getAccuracy(False)
print ("Pre-attack Accuracy (robust classifier): " + str(preAccuracy2))
print ("Post-attack Accuracy: (robust classifier): " + str(postAccuracy2))

In [ ]:
metrics2.plotROC(True)